In [54]:
import pickle
import sys
import math
import numpy as np
import pandas as pd


import json
import reverse_geocoder as rg
import pymorphy2
import re
from tqdm import tqdm


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

In [2]:
#nec funct
def rmsle(actual, predicted):
    return np.sqrt(
            np.mean(
                (np.power
                 (np.log(np.array(actual)+1) - 
                  np.log(np.array(predicted)+1), 
                  2)
                )
            )
    )

In [4]:
with open("datagym-ml-recap-mf/train_sample.pckl/train_sample.pckl", "rb") as pkl:
    data_set = pickle.load(pkl)

In [5]:
data_set.shape

(100000, 17)

In [6]:
sml = data_set.copy().head(100).reset_index()
sml.drop('index', axis = 1, inplace = True)

In [7]:
sml.head()

,can_buy,can_promote,category,contacts_visible,date_created,delivery_available,description,fields,id,images,location,mortgage_available,name,payment_available,price,subcategory,subway
0,False,False,9,True,1492780671,False,,"[{'field': {'name': 'Женский гардероб', 'id': ...",3edeb34cf93f490ff760af85,"[{'id': '58fa06746c86cb4f22313452', 'num': 1, ...","{'latitude': 55.806888, 'longitude': 37.546077}",False,Сумка DG,True,199900,914,NaN
1,False,False,22,True,1476824319,False,8-12 лет,"[{'field': {'name': 'Детский гардероб', 'id': ...",c98febd50dad3cc0ffc86085,"[{'id': '58068ccc04559f59bdbda92d', 'num': 1, ...","{'latitude': 55.692979, 'longitude': 37.872337}",False,Комплект,False,35000,2202,NaN
2,False,False,22,True,1473004313,False,"На девочку 1,5 г,состояние хорошее","[{'field': {'name': 'Детский гардероб', 'id': ...",ade01e13912a46a99134cc75,"[{'id': '57cc42ecd53f3dcf17dc01c8', 'num': 1, ...","{'latitude': 55.639011, 'longitude': 37.349378}",False,Пальтишко демисезонное,False,30000,2204,NaN
3,False,False,22,True,1476307221,False,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...","[{'field': {'name': 'Детский гардероб', 'id': ...",ab3e6941c11304c1519aef75,"[{'id': '580546528ae74be97723532e', 'num': 1, ...","{'latitude': 55.847334, 'longitude': 37.495834}",False,Attipas,True,80000,2209,NaN
4,False,False,22,True,1503487787,False,,"[{'field': {'name': 'Детский гардероб', 'id': ...",252452a91c944a22c276d995,"[{'id': '599d66f3f235022f7411a535', 'num': 1, ...","{'latitude': 60.044826, 'longitude': 30.35546}",False,Жилет теплый,True,50000,2204,NaN


In [8]:
def feature_process(df):
    
    def count_filled(tp):
        basics = ['Цвет', 'Пол', 'Тип']        
        cnt = 0
        for j in tqdm(tp[0]):
            try:
                if j['field']['name'] in basics:
                    cnt +=1
            except:
                pass
        return cnt/len(tp)
    
    def getMeaningfullWords(text):
        words=[]
        tokens=re.findall('[А-Яа-яЁёA-Za-z]+\-[А-Яа-яЁёA-Za-z]+|[А-Яа-яЁёA-Za-z]+', str(text))
        for t in tokens:
            pv=morph.parse(t)
            for p in pv:
                if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                    words.append(p.normal_form)
                    break
        return words

    def getMeaningfullString(txt):
        res = ' '.join(getMeaningfullWords(txt))
        return res
    
    def get_place(loc):
        loc_list = [loc['latitude'], loc['longitude']]
        result = rg.search(loc_list)[0]
        return result['name'], result['admin1'], result['cc']
    
    tr_feature = ['can_buy', 'can_promote', 'contacts_visible', 'delivery_available',
                  'mortgage_available', 'payment_available', 
                 ]
    
    
    
    to_drop = ['date_created', 'fields', 'id', 'images', 'location', 'name', 
               'description',
               #'district_non_parced'#, 
              ]
    
    ###########
    # true/false => 1/0
    for f in tr_feature:
        #True/False на 1 / 0
        df[f] = df[f].map({False: 0, True: 1})
    
    print('Обработка True/False')
    
    #counting imgs
    df['cnt_images'] = df['images'].map(lambda x: int(x[0]['num']) if x[0]['num'] else 0)
    
    #почистим описание от довйных пробелов:
    df['description'] = df['description'].map(lambda x: x.lower().strip().replace("  ", " "))
    
    #   заменим разные варианты написаня одних слов:
    df['name'] = df['name'].map(lambda x:x.lower().strip().replace('айфон', 'iphone').replace('самсунг', 'samsung'))

    #  найдем всё бесплатное и запишем в отдельную колонку:
    df['free'] = np.where(
        ((df['description'].str.contains("почти за") == False) &
        (df['description'].str.contains("почти даром") == False) &
        (df['description'].str.contains("почти бесплатно") == False) &
        (df['description'].str.contains("даром за") == False) &
        (df['description'].str.contains("прода") == False) &
        (df['description'].str.contains("бесплатно привезу") == False) &
        (df['description'].str.contains("недорого") == False) &
        (df['description'].str.contains("практически") == False) &
        (df['description'].str.contains("нуждающим") == False) &
        (df['description'].str.contains("можно сказать") == False) &
        (df['description'].str.contains("цена") == False) &
        (df['description'].str.contains("считай") == False) &
        (df['description'].str.contains("при заказе") == False) &
        (df['description'].str.contains("при покупке") == False) &
        (df['description'].str.contains(" символ") == False) &
        (df['description'].str.contains("чехол") == False) &
        (df['description'].str.contains("чехлы") == False) &
        (df['description'].str.contains("монтаж") == False) &
        (df['description'].str.contains("установк") == False) &
        (df['description'].str.contains("дешевле только") == False) &

        (df['name'].str.contains("прода") == False) &
        (df['name'].str.contains("почти") == False) &
        (df['name'].str.contains("считай") == False)
        ) & 
        (
            (
                (
                    (df['name'].str.contains("даром")) |
                    (df['name'].str.contains("бесплатно"))
                ) &
                (df['name'].str.contains("достав") == False) &
                (df['name'].str.contains("привез") == False)
            ) | (
                (
                    (df['description'].str.contains("бесплатно")) |
                    (df['description'].str.contains("даром"))
                ) &
                (df['description'].str.contains("достав") == False) &
                (df['description'].str.contains("привез") == False)
            )
        ), 1, 0)
    #name filed disc
    df['fielded_names'] = count_filled(df['fields'])
    df['fielded_names_main'] = df['fields'].map(lambda x: x[0]['field']['name'] if x[0]['field']['name'] else 'нет')
    df['fielded_names_dop'] = df['fields'].map(lambda x: x[1]['field']['name'] if x[1]['field']['name'] else 'нет')
    
    print('Нормализовано описание')
    
    morph=pymorphy2.MorphAnalyzer()
    df['clear_description'] = df['description'].apply(func = getMeaningfullString)

    #заменим описание на его длину:
    df['description_len'] = df['description'].map(lambda x: len(x))
    df['clear_description_len'] = df['clear_description'].map(lambda x: len(x.split()))
    
    print('Обработка описания')
    
    # парсинг координат
    #df['district_non_parced'] = df['location'].apply(get_place)
    #df['district'] = df['district_non_parced'].map(lambda x: x[0])
    #df['city'] = df['district_non_parced'].map(lambda x: x[1])  
    #df['country'] = df['district_non_parced'].map(lambda x: x[2])  
    
    print('Обработка координат')
    
    df.fillna(0, inplace = True)
    df.drop(to_drop, axis = 1, inplace = True)
    
    df['price'] = df['price'].map(lambda x: x if x >= 0 else 0)
    return df

In [9]:
#get
tmp = feature_process(sml.copy())

#OHE
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(pd.DataFrame(tmp['fielded_names_main']))
main_cats = pd.DataFrame(data = enc.transform(pd.DataFrame(tmp['fielded_names_main'])).toarray(),
                       columns = enc.categories_[0])

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(pd.DataFrame(tmp['fielded_names_dop']))
main_cats = pd.DataFrame(data = enc.transform(pd.DataFrame(tmp['fielded_names_dop'])).toarray(),
                       columns = enc.categories_[0])

tmp = pd.concat([tmp, main_cats], axis = 1)


tmp.head()

100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 3998.38it/s]

Обработка True/False
Нормализовано описание


Обработка описания
Обработка координат


,can_buy,can_promote,category,contacts_visible,delivery_available,mortgage_available,payment_available,price,subcategory,subway,...,Столы и стулья,Текстиль и ковры,Товары для мам,Тренажеры и фитнес,Уход за волосами,Футболки и топы,Холодильники,Шкафы и комоды,Штаны и шорты,Электроинструменты
0,0,0,9,1,0,0,1,199900,914,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,22,1,0,0,0,35000,2202,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,22,1,0,0,0,30000,2204,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,22,1,0,0,1,80000,2209,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,22,1,0,0,1,50000,2204,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
tmp.columns

Index(['can_buy', 'can_promote', 'category', 'contacts_visible',
       'delivery_available', 'mortgage_available', 'payment_available',
       'price', 'subcategory', 'subway', 'cnt_images', 'free', 'fielded_names',
       'fielded_names_main', 'fielded_names_dop', 'clear_description',
       'description_len', 'clear_description_len', 'Аксессуары',
       'Будущим мамам', 'Верхняя одежда', 'Водные виды спорта',
       'Другие животные', 'Другое', 'Запчасти', 'Зимние виды спорта',
       'Игровые приставки', 'Игрушки и игры', 'Книги и журналы',
       'Колеса, колпаки и камеры', 'Коллекционирование', 'Коляски',
       'Комбинезоны и боди', 'Комплектующие и запчасти', 'Компьютеры',
       'Кормление и питание', 'Кошки', 'Красота и здоровье',
       'Медицинские товары', 'Мобильные телефоны', 'Наушники', 'Нижнее белье',
       'Обувь', 'Обустройство детской', 'Оформление интерьера', 'Парфюмерия',
       'Пиджаки и костюмы', 'Платья и юбки', 'Посуда',
       'Приготовление напитков', 'Св

In [11]:

cts = ['can_buy', 'can_promote', 'category', 'contacts_visible',
'delivery_available', 'mortgage_available', 'payment_available',
'subcategory', 'subway', 'free','Аксессуары',
'Будущим мамам', 'Верхняя одежда', 'Водные виды спорта',
'Другие животные', 'Другое', 'Запчасти', 'Зимние виды спорта',
'Игровые приставки', 'Игрушки и игры', 'Книги и журналы',
'Колеса, колпаки и камеры', 'Коллекционирование', 'Коляски',
'Комбинезоны и боди', 'Комплектующие и запчасти', 'Компьютеры',
'Кормление и питание', 'Кошки', 'Красота и здоровье',
'Медицинские товары', 'Мобильные телефоны', 'Наушники', 'Нижнее белье',
'Обувь', 'Обустройство детской', 'Оформление интерьера', 'Парфюмерия',
'Пиджаки и костюмы', 'Платья и юбки', 'Посуда',
'Приготовление напитков', 'Свитеры и толстовки', 'Спецодежда',
'Спортивная одежда', 'Стиральные машины', 'Столы и стулья',
'Текстиль и ковры', 'Товары для мам', 'Тренажеры и фитнес',
'Уход за волосами', 'Футболки и топы', 'Холодильники', 'Шкафы и комоды',
'Штаны и шорты', 'Электроинструменты']

#cats

In [12]:
from catboost import CatBoostRegressor, Pool

In [50]:
train = tmp.iloc[:70]
test = tmp.iloc[70:]

In [51]:
not_to_train =  ['fielded_names', 'fielded_names_main', 
                 'fielded_names_dop', 'clear_description', 'price']

x = train.drop(not_to_train, axis = 1)#.values
print(x.shape)

cats = list()
for i in range(len(x.columns)):
    if x.columns[i] in cts:
        cats.append(i)

y = np.log(train['price'], where=(train['price'].values >0))

train_data = Pool(data=x.values,
                  label=y.values)

model = CatBoostRegressor()

model.fit(train_data)
preds_class = model.predict(test.drop(not_to_train, axis = 1).values)

metric = rmsle(np.log(test['price'], where=(test['price'].values>0)), preds_class)
print('RMSLE - ', metric)

(70, 59)
0:	learn: 1.7022235	total: 7.4ms	remaining: 7.39s
1:	learn: 1.6928063	total: 16.1ms	remaining: 8.02s
2:	learn: 1.6872336	total: 24.8ms	remaining: 8.26s
3:	learn: 1.6738885	total: 33.3ms	remaining: 8.29s
4:	learn: 1.6729750	total: 42.7ms	remaining: 8.49s
5:	learn: 1.6625881	total: 52.6ms	remaining: 8.72s
6:	learn: 1.6523907	total: 64.1ms	remaining: 9.1s
7:	learn: 1.6442529	total: 73.4ms	remaining: 9.1s
8:	learn: 1.6340737	total: 81.7ms	remaining: 9s
9:	learn: 1.6204313	total: 90.4ms	remaining: 8.95s
10:	learn: 1.6126912	total: 97.4ms	remaining: 8.76s
11:	learn: 1.6047065	total: 106ms	remaining: 8.7s
12:	learn: 1.5970932	total: 118ms	remaining: 8.94s
13:	learn: 1.5874823	total: 128ms	remaining: 9.01s
14:	learn: 1.5806588	total: 136ms	remaining: 8.96s
15:	learn: 1.5742961	total: 145ms	remaining: 8.9s
16:	learn: 1.5669693	total: 153ms	remaining: 8.85s
17:	learn: 1.5610928	total: 162ms	remaining: 8.81s
18:	learn: 1.5601285	total: 170ms	remaining: 8.77s
19:	learn: 1.5518633	total: 1

176:	learn: 1.0890278	total: 1.66s	remaining: 7.74s
177:	learn: 1.0854326	total: 1.67s	remaining: 7.73s
178:	learn: 1.0851139	total: 1.69s	remaining: 7.73s
179:	learn: 1.0827938	total: 1.7s	remaining: 7.73s
180:	learn: 1.0819638	total: 1.71s	remaining: 7.72s
181:	learn: 1.0808563	total: 1.72s	remaining: 7.71s
182:	learn: 1.0782124	total: 1.72s	remaining: 7.69s
183:	learn: 1.0741637	total: 1.73s	remaining: 7.68s
184:	learn: 1.0727346	total: 1.74s	remaining: 7.67s
185:	learn: 1.0710068	total: 1.75s	remaining: 7.65s
186:	learn: 1.0705049	total: 1.76s	remaining: 7.64s
187:	learn: 1.0688103	total: 1.76s	remaining: 7.63s
188:	learn: 1.0677932	total: 1.77s	remaining: 7.61s
189:	learn: 1.0658267	total: 1.78s	remaining: 7.6s
190:	learn: 1.0651955	total: 1.79s	remaining: 7.58s
191:	learn: 1.0607821	total: 1.8s	remaining: 7.57s
192:	learn: 1.0588372	total: 1.81s	remaining: 7.56s
193:	learn: 1.0582411	total: 1.81s	remaining: 7.54s
194:	learn: 1.0580251	total: 1.82s	remaining: 7.53s
195:	learn: 1.0

337:	learn: 0.8152480	total: 3.09s	remaining: 6.05s
338:	learn: 0.8141043	total: 3.1s	remaining: 6.05s
339:	learn: 0.8125016	total: 3.11s	remaining: 6.04s
340:	learn: 0.8121591	total: 3.12s	remaining: 6.03s
341:	learn: 0.8116676	total: 3.13s	remaining: 6.02s
342:	learn: 0.8114712	total: 3.14s	remaining: 6.01s
343:	learn: 0.8105105	total: 3.15s	remaining: 6.01s
344:	learn: 0.8096469	total: 3.16s	remaining: 6s
345:	learn: 0.8090911	total: 3.17s	remaining: 5.99s
346:	learn: 0.8082415	total: 3.17s	remaining: 5.97s
347:	learn: 0.8073353	total: 3.18s	remaining: 5.96s
348:	learn: 0.8063843	total: 3.19s	remaining: 5.95s
349:	learn: 0.8051962	total: 3.2s	remaining: 5.94s
350:	learn: 0.8041575	total: 3.21s	remaining: 5.93s
351:	learn: 0.8032386	total: 3.22s	remaining: 5.92s
352:	learn: 0.8018603	total: 3.22s	remaining: 5.91s
353:	learn: 0.8009920	total: 3.23s	remaining: 5.9s
354:	learn: 0.7990850	total: 3.24s	remaining: 5.89s
355:	learn: 0.7986965	total: 3.25s	remaining: 5.88s
356:	learn: 0.7980

500:	learn: 0.6846508	total: 4.52s	remaining: 4.5s
501:	learn: 0.6844919	total: 4.53s	remaining: 4.49s
502:	learn: 0.6839807	total: 4.54s	remaining: 4.49s
503:	learn: 0.6837966	total: 4.55s	remaining: 4.48s
504:	learn: 0.6827926	total: 4.56s	remaining: 4.47s
505:	learn: 0.6826480	total: 4.57s	remaining: 4.46s
506:	learn: 0.6824746	total: 4.58s	remaining: 4.45s
507:	learn: 0.6823810	total: 4.58s	remaining: 4.44s
508:	learn: 0.6819326	total: 4.59s	remaining: 4.43s
509:	learn: 0.6802384	total: 4.6s	remaining: 4.42s
510:	learn: 0.6798249	total: 4.61s	remaining: 4.41s
511:	learn: 0.6796306	total: 4.62s	remaining: 4.4s
512:	learn: 0.6788555	total: 4.63s	remaining: 4.39s
513:	learn: 0.6779089	total: 4.63s	remaining: 4.38s
514:	learn: 0.6778124	total: 4.64s	remaining: 4.37s
515:	learn: 0.6772909	total: 4.65s	remaining: 4.36s
516:	learn: 0.6771800	total: 4.66s	remaining: 4.35s
517:	learn: 0.6755149	total: 4.67s	remaining: 4.34s
518:	learn: 0.6753109	total: 4.67s	remaining: 4.33s
519:	learn: 0.6

663:	learn: 0.6174655	total: 5.94s	remaining: 3s
664:	learn: 0.6171676	total: 5.95s	remaining: 3s
665:	learn: 0.6170581	total: 5.96s	remaining: 2.99s
666:	learn: 0.6165574	total: 5.97s	remaining: 2.98s
667:	learn: 0.6164720	total: 5.98s	remaining: 2.97s
668:	learn: 0.6163013	total: 5.99s	remaining: 2.96s
669:	learn: 0.6127347	total: 6s	remaining: 2.96s
670:	learn: 0.6126514	total: 6.01s	remaining: 2.95s
671:	learn: 0.6124477	total: 6.02s	remaining: 2.94s
672:	learn: 0.6114796	total: 6.03s	remaining: 2.93s
673:	learn: 0.6113563	total: 6.03s	remaining: 2.92s
674:	learn: 0.6109944	total: 6.04s	remaining: 2.91s
675:	learn: 0.6108023	total: 6.05s	remaining: 2.9s
676:	learn: 0.6105911	total: 6.06s	remaining: 2.89s
677:	learn: 0.6102567	total: 6.07s	remaining: 2.88s
678:	learn: 0.6101740	total: 6.08s	remaining: 2.87s
679:	learn: 0.6100105	total: 6.08s	remaining: 2.86s
680:	learn: 0.6099152	total: 6.09s	remaining: 2.85s
681:	learn: 0.6097828	total: 6.1s	remaining: 2.84s
682:	learn: 0.6086115	t

825:	learn: 0.5668023	total: 7.37s	remaining: 1.55s
826:	learn: 0.5661873	total: 7.38s	remaining: 1.54s
827:	learn: 0.5657978	total: 7.39s	remaining: 1.53s
828:	learn: 0.5657115	total: 7.4s	remaining: 1.53s
829:	learn: 0.5641836	total: 7.41s	remaining: 1.52s
830:	learn: 0.5641375	total: 7.42s	remaining: 1.51s
831:	learn: 0.5640108	total: 7.43s	remaining: 1.5s
832:	learn: 0.5638607	total: 7.43s	remaining: 1.49s
833:	learn: 0.5628905	total: 7.44s	remaining: 1.48s
834:	learn: 0.5625018	total: 7.45s	remaining: 1.47s
835:	learn: 0.5620706	total: 7.46s	remaining: 1.46s
836:	learn: 0.5619024	total: 7.47s	remaining: 1.45s
837:	learn: 0.5617719	total: 7.48s	remaining: 1.45s
838:	learn: 0.5617657	total: 7.49s	remaining: 1.44s
839:	learn: 0.5613217	total: 7.49s	remaining: 1.43s
840:	learn: 0.5608516	total: 7.5s	remaining: 1.42s
841:	learn: 0.5607850	total: 7.51s	remaining: 1.41s
842:	learn: 0.5605739	total: 7.52s	remaining: 1.4s
843:	learn: 0.5604703	total: 7.53s	remaining: 1.39s
844:	learn: 0.56

989:	learn: 0.5210715	total: 8.8s	remaining: 88.9ms
990:	learn: 0.5208501	total: 8.81s	remaining: 80ms
991:	learn: 0.5207158	total: 8.82s	remaining: 71.1ms
992:	learn: 0.5206543	total: 8.83s	remaining: 62.2ms
993:	learn: 0.5206281	total: 8.84s	remaining: 53.3ms
994:	learn: 0.5199019	total: 8.85s	remaining: 44.5ms
995:	learn: 0.5197887	total: 8.86s	remaining: 35.6ms
996:	learn: 0.5194810	total: 8.87s	remaining: 26.7ms
997:	learn: 0.5193475	total: 8.87s	remaining: 17.8ms
998:	learn: 0.5192911	total: 8.88s	remaining: 8.89ms
999:	learn: 0.5190662	total: 8.89s	remaining: 0us
RMSLE -  0.16543586118866208


In [33]:
np.where(preds_class < 0, )
#rmsle(test['price'].values, preds_class)

(array([12, 29], dtype=int64),)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


nan

In [41]:
tfidf = TfidfVectorizer()
response = tfidf.fit_transform(tmp['description'].values)
feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    print (feature_names[col], ' - ', response[0, col])

12  -  0.0
лет  -  0.0
на  -  0.0
девочку  -  0.0
состояние  -  0.0
хорошее  -  0.0
размер  -  0.0
135mm  -  0.0
euro  -  0.0
22  -  0.0
прочная  -  0.0
мягкая  -  0.0
не  -  0.0
скользящая  -  0.0
обувь  -  0.0
для  -  0.0
маленьких  -  0.0
гибкая  -  0.0
подошва  -  0.0
обеспечивает  -  0.0
много  -  0.0
места  -  0.0
движения  -  0.0
пальчиков  -  0.0
производится  -  0.0
без  -  0.0
использования  -  0.0
клея  -  0.0
токсичных  -  0.0
веществ  -  0.0
благодаря  -  0.0
небольшим  -  0.0
отверстиям  -  0.0
подошве  -  0.0
позволяет  -  0.0
дышать  -  0.0
ножке  -  0.0
машинная  -  0.0
стирка  -  0.0
быстро  -  0.0
сохнут  -  0.0
состав  -  0.0
материал  -  0.0
подошвы  -  0.0
термопластичный  -  0.0
силикон  -  0.0
100  -  0.0
носочка  -  0.0
хлопок  -  0.0
80  -  0.0
спаднекс  -  0.0
17  -  0.0
резиновая  -  0.0
нить  -  0.0
полиуритановые  -  0.0
нити  -  0.0
помогает  -  0.0
правильному  -  0.0
моторному  -  0.0
развитию  -  0.0


In [52]:
np.log(test['price'], where=(test['price'].values>0))

70    12.206073
71    11.407565
72    10.714418
73    10.463103
74    11.512925
75    10.463103
76    13.710150
77    13.304685
78     9.615805
79    10.308953
80    13.815511
81    11.289782
82     4.605170
83    12.206073
84    10.596635
85    13.304685
86    11.512925
87    10.596635
88     9.615805
89    11.512925
90     8.987197
91    10.714418
92    10.571317
93    10.239960
94     8.987197
95     9.210340
96     9.104980
97    10.819778
98    10.714418
99     9.615805
Name: price, dtype: float64

In [53]:
2.1828 ** np.log(test['price'], where=(test['price'].values>0))

70    13741.332294
71     7367.558579
72     4288.812276
73     3524.828096
74     7999.121283
75     3524.828096
76    44456.170961
77    32394.682516
78     1819.249027
79     3125.206446
80    48267.047965
81     6720.374976
82       36.409684
83    13741.332294
84     3912.072960
85    32394.682516
86     7999.121283
87     3912.072960
88     1819.249027
89     7999.121283
90     1113.743160
91     4288.812276
92     3835.516504
93     2961.346089
94     1113.743160
95     1325.665107
96     1220.998530
97     4656.458336
98     4288.812276
99     1819.249027
Name: price, dtype: float64

In [57]:
rmsle(test['price'], math.e ** preds_class)

1.6226010717400001

In [58]:
https://github.com/katerynaCh/Predicting-Housing-Prices/blob/master/prediction.ipynb

SyntaxError: invalid syntax (<ipython-input-58-9e062d17a2c7>, line 1)

In [59]:
https://github.com/yixinsunn/restaurant-visitor-forecasting/blob/master/train_procedure1.ipynb


SyntaxError: invalid syntax (<ipython-input-59-e8cfbc5b59c7>, line 1)